In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

TensorFlow 2.x selected.
Found GPU at: /device:GPU:0


In [3]:
cd /content/drive/My Drive/Colab Notebooks/Santander

/content/drive/My Drive/Colab Notebooks/Santander


In [4]:
!ls

ANN_Santander.ipynb  Santander_Customer_Satisfaction.ipynb  test.csv
Santander_2.ipynb    Santander_v1.ipynb			    train.csv


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None, 'display.max_rows', 100)

In [6]:
df = pd.read_csv('train.csv')
X_test = pd.read_csv('test.csv')
df.shape, X_test.shape

((76020, 371), (75818, 370))

In [7]:
#Splitting into Train and Dev Set
from sklearn.model_selection import train_test_split
X_train, X_dev, y_train, y_dev = train_test_split(df.iloc[:,:-1], df.iloc[:,-1] , test_size=0.05, random_state = 0)
X_train.shape, X_dev.shape, y_train.shape, y_dev.shape,

((72219, 370), (3801, 370), (72219,), (3801,))

In [0]:
features = ['num_var8_0', 'num_var13', 'imp_op_var39_ult1', 'num_op_var39_efect_ult3', 'saldo_medio_var12_ult3', 'num_var42_0', 'ind_var9_ult1', 'num_op_var41_efect_ult3', 'num_op_var39_comer_ult3', 'num_var24', 'num_var24_0', 'num_var13_corto', 'ind_var13', 'imp_op_var39_efect_ult3', 'ind_var10cte_ult1', 'imp_op_var41_comer_ult1', 'ind_var24', 'num_op_var39_ult3', 'num_var41_0', 'num_var13_0', 'saldo_var42', 'num_med_var22_ult3', 'num_var5_0', 'ind_var13_corto', 'ind_var13_corto_0', 'num_var37_0', 'num_meses_var13_corto_ult3', 'num_var5', 'ind_var37_0', 'num_var45_ult3', 'num_op_var41_comer_ult1', 'ind_var9_cte_ult1', 'num_var13_corto_0', 'num_meses_var12_ult3', 'saldo_medio_var13_corto_ult3', 'saldo_var24', 'num_op_var39_comer_ult1', 'ind_var41_0', 'num_op_var39_ult1', 'num_op_var39_efect_ult1', 'num_op_var39_hace2', 'imp_op_var41_comer_ult3', 'num_var45_hace2', 'num_op_var41_ult3', 'num_var35', 'imp_op_var39_efect_ult1', 'num_meses_var5_ult3', 'saldo_medio_var13_corto_ult1', 'num_var12', 'num_op_var41_comer_ult3', 'saldo_medio_var12_ult1']

In [9]:
len(features)

51

In [0]:
X_train = X_train[features]
X_dev = X_dev[features]

In [11]:
from imblearn.over_sampling import SMOTE
X_resample, y_resample = SMOTE().fit_sample(X_train,y_train)
X_resample.shape, y_resample.shape

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


((138754, 51), (138754,))

# Build the ANN

In [0]:
#Scaling
from sklearn.preprocessing import StandardScaler
scaller = StandardScaler()
X_resample = scaller.fit_transform(X_resample)
X_dev = scaller.transform(X_dev)

In [0]:
# # Should I scale the dependent variable as well? Why should I
# sc_y = StandardScaler()
# y_resample = sc_y.fit_transform(y_resample)

In [14]:
N, D = X_resample.shape
N, D

(138754, 51)

In [0]:
# Build the model
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(64, input_shape=(D,), activation='relu'),
  tf.keras.layers.Dense(32, input_shape=(D,), activation='tanh'),
  tf.keras.layers.Dense(32, input_shape=(D,), activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

In [0]:
# Compile and fit
opt = tf.keras.optimizers.Adam()
model.compile(optimizer=opt, 
              loss='binary_crossentropy',
              metrics=['accuracy'])

r = model.fit(X_resample, y_resample, validation_data=(X_dev, y_dev), epochs=50)

Train on 138754 samples, validate on 3801 samples
Epoch 1/50
138754/138754 [==============================] - 15s 110us/sample - loss: 0.5637 - accuracy: 0.7271 - val_loss: 0.5295 - val_accuracy: 0.7345
Epoch 2/50
138754/138754 [==============================] - 13s 94us/sample - loss: 0.5297 - accuracy: 0.7483 - val_loss: 0.5569 - val_accuracy: 0.7130
Epoch 3/50
138754/138754 [==============================] - 13s 94us/sample - loss: 0.5136 - accuracy: 0.7567 - val_loss: 0.5071 - val_accuracy: 0.7403
Epoch 4/50
138754/138754 [==============================] - 13s 92us/sample - loss: 0.5051 - accuracy: 0.7615 - val_loss: 0.5879 - val_accuracy: 0.7059
Epoch 5/50
138754/138754 [==============================] - 13s 94us/sample - loss: 0.4987 - accuracy: 0.7642 - val_loss: 0.5166 - val_accuracy: 0.7419
Epoch 6/50
138754/138754 [==============================] - 13s 94us/sample - loss: 0.4934 - accuracy: 0.7679 - val_loss: 0.5063 - val_accuracy: 0.7443
Epoch 7/50
138754/138754 [===========

In [0]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 64)                3328      
_________________________________________________________________
dense_8 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_9 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 33        
Total params: 6,497
Trainable params: 6,497
Non-trainable params: 0
_________________________________________________________________


In [0]:
predictions = model.predict(X_test[features])

In [0]:
predictions_rounded = np.round(predictions).flatten()
predictions

array([[5.00679016e-06],
       [1.99675560e-06],
       [0.00000000e+00],
       ...,
       [5.87105751e-05],
       [1.00488405e-05],
       [5.87080431e-05]], dtype=float32)

In [0]:
np.shape(predictions)

(75818, 1)

In [0]:
# Extracting the result ready to submit
test_id = pd.read_csv('test.csv').iloc[:,0]
submission = pd.DataFrame({"ID":test_id, "TARGET":predictions.flatten()})
submission.to_csv("submission_v12.csv", index=False)